In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
from tensorflow.keras.models import load_model

saved_dir = 'D:/JATHURSH/Jupyter_notebook_projects/DistractionDetectFinal/saved_data/models/'

# load model
model = load_model(saved_dir + 'dis_det_model.h5')

In [3]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow import lite

# Convert the Keras model to TensorFlow Lite with float16 precision
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.target_spec.supported_types = [tf.float16]
converter._experimental_lower_tensor_list_ops = False
tflite_fp16_model = converter.convert()

# Load the converted TensorFlow Lite model with float16 precision
interpreter = tf.lite.Interpreter(model_content=tflite_fp16_model)
interpreter.allocate_tensors()

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpuw45fntj\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpuw45fntj\assets


In [6]:
from pygame import mixer

# Initialize the mixer module
mixer.init()

alarm_sound_path = 'D:/JATHURSH/Jupyter_notebook_projects/DistractionDetectFinal/saved_data/test_assets/alarm.mp3'

# Load the alarm sound
mixer.music.load(alarm_sound_path)

# # Play the alarm sound
# mixer.music.play()

# # Wait for a few seconds (you can adjust the duration)
# time.sleep(2)

# # Stop the alarm sound
# mixer.music.stop()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
import time

# Define class labels
class_labels = {
    0: 'Normal Driving',
    1: 'Texting',
    2: 'Talking on the Phone',
    3: 'Operating the Radio',
    4: 'Drinking',
    5: 'Reaching Behind',
    6: 'Hair and Makeup',
    7: 'Talking to Passenger'
}

# Open a video stream
cap = cv2.VideoCapture('D:/JATHURSH/Jupyter_notebook_projects/DistractionDetectionWithDA/saved_data/test_videos/test_vdo_distraction_detect.mp4')

# Set the desired frame rate
your_frame_rate = 30  # Adjust this value based on your video's frame rate
cap.set(cv2.CAP_PROP_FPS, your_frame_rate)

# Set the font properties
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_thickness = 2

warning_duration_threshold = 1.5

# Initialize variables to track distractions and their durations
current_distraction = None
current_distraction_start_time = None

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Resize the frame to match the model's expected sizing
    resized_frame = cv2.resize(frame, (224, 224))

    # Convert the BGR image to RGB
    frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

    # Expand dimensions to match model input shape
    frame_expanded = np.expand_dims(frame_rgb, axis=0)

    # Preprocess the frame for the model input
    frame_preprocessed = preprocess_input(frame_expanded)

    # Set the input tensor for the model
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], frame_preprocessed)

    # Run inference
    interpreter.invoke()

    # Get the output tensor
    predictions = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

    # Get the predicted class index and label
    predicted_class_index = np.argmax(predictions)
    predicted_class_label = class_labels[predicted_class_index]

    # Display the frame with the predicted class label
    cv2.putText(frame, f'{predicted_class_index} ({predicted_class_label})', (10, 30),
                font, font_scale, (0, 255, 0), font_thickness)

    # Check for specific distractions and set warnings
    if predicted_class_label != 'Normal Driving':
        # Distraction is not 'Normal Driving'
        if predicted_class_label == current_distraction:
            # Same distraction as the previous frame
            elapsed_time = time.time() - current_distraction_start_time

            if elapsed_time >= warning_duration_threshold:
                # Warning condition satisfied
                cv2.putText(frame, f'WARNING: {current_distraction} for {elapsed_time:.1f} seconds!',
                            (10, 60), font, font_scale, (0, 0, 255), font_thickness)
                
                # trigger alarm
                # Check if the alarm is not already playing
                if not mixer.music.get_busy():
                    # Play the alarm sound
                    mixer.music.play()

        else:
            # Distraction has changed, reset the tracking variables
            current_distraction = predicted_class_label
            current_distraction_start_time = time.time()
            
            # Stop the alarm sound
            mixer.music.stop()
    
    cv2.imshow('Distraction Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()